## Subprocess management

The subprocess module allows you to spawn new processes, connect to their input/output/error pipes, and obtain their return codes.   
This module intends to replace several older modules and functions:
###### os.system  
###### os.spawn*  
###### os.popen*  
###### popen2.*  
###### commands.*  
Information about how this module can be used to replace the older functions can be found in the subprocess-replacements section.  

###### POSIX users (Linux, BSD, etc.) are strongly encouraged to install and use the much more recent subprocess32 module instead of the version included with python 2.7. It is a drop in replacement with better behavior in many situations.

### Using the subprocess Module

The recommended way to launch subprocesses is to use the following convenience functions.  
For more advanced use cases when these do not meet your needs, use the underlying Popen interface.

###### subprocess.call(args, *, stdin=None, stdout=None, stderr=None, shell=False)
Run the command described by args.   
Wait for command to complete, then return the returncode attribute.

The arguments shown above are merely the most common ones, described below in Frequently Used Arguments (hence the slightly odd notation in the abbreviated signature).   

The full function signature is the same as that of the Popen constructor - this functions passes all supplied arguments directly through to that interface.

In [9]:
import subprocess as sp
sp.call(['ls', '-lh'])

0

In [10]:
sp.call('df -lh', shell=True)

0

Warning:  
Using shell=True can be a security hazard.   
See the warning under Frequently Used Arguments for details.  

Note:   
Do not use stdout=PIPE or stderr=PIPE with this function as that can deadlock based on the child process output volume.   
Use Popen with the communicate() method when you need pipes.

###### subprocess.check_call(args, *, stdin=None, stdout=None, stderr=None, shell=False)
Run command with arguments.   
Wait for command to complete.   
If the return code was zero then return, otherwise raise CalledProcessError.   
The CalledProcessError object will have the return code in the returncode attribute.

In [12]:
sp.check_call(["ls", "-lh"])

0

In [13]:
sp.check_call("exit 1", shell=True)

CalledProcessError: Command 'exit 1' returned non-zero exit status 1

###### subprocess.check_output(args, *, stdin=None, stderr=None, shell=False, universal_newlines=False)
Run command with arguments and return its output as a byte string.   
If the return code was non-zero it raises a CalledProcessError.   
The CalledProcessError object will have the return code in the returncode attribute and any output in the output attribute.

In [15]:
sp.check_output(["cat", '/home/liheyi/ip.txt'])

'192.168.112.108\n192.168.112.109\n192.168.112.110\n192.168.112.111\n'

In [16]:
sp.check_output("cat file", shell=True)

CalledProcessError: Command 'cat file' returned non-zero exit status 1

To also capture standard error in the result, use stderr=subprocess.STDOUT:

In [19]:
sp.check_output(
    "ls non_existent_file; exit 0",
    stderr=sp.STDOUT,
    shell=True)

'ls: cannot access non_existent_file: No such file or directory\n'

###### subprocess.PIPE
Special value that can be used as the stdin, stdout or stderr argument to Popen and indicates that a pipe to the standard stream should be opened.

###### subprocess.STDOUT
Special value that can be used as the stderr argument to Popen and indicates that standard error should go into the same handle as standard output.

###### exception subprocess.CalledProcessError(retcode, cmd, output=output)
Exception raised when a process run by check_call() or check_output() returns a non-zero exit status.

returncode
Exit status of the child process.  

cmd  
Command that was used to spawn the child process.  

output  
Output of the child process if this exception is raised by check_output(). Otherwise, None.


####  Frequently Used Arguments

To support a wide variety of use cases, the Popen constructor (and the convenience functions) accept a large number of optional arguments.   
For most typical use cases, many of these arguments can be safely left at their default values.   
The arguments that are most commonly needed are:

###### args
args is required for all calls and should be a string, or a sequence of program arguments.  

Providing a sequence of arguments is generally preferred, as it allows the module to take care of any required escaping and quoting of arguments (e.g. to permit spaces in file names).   
If passing a single string, either shell must be True (see below) or else the string must simply name the program to be executed without specifying any arguments.

###### stdin, stdout and stderr
stdin, stdout and stderr specify the executed program’s standard input, standard output and standard error file handles, respectively.   
Valid values are PIPE, an existing file descriptor (a positive integer), an existing file object, and None.  
PIPE indicates that a new pipe to the child should be created.   
With the default settings of None, no redirection will occur; the child’s file handles will be inherited from the parent.   
Additionally, stderr can be STDOUT, which indicates that the stderr data from the child process should be captured into the same file handle as for stdout.  

When stdout or stderr are pipes and universal_newlines is True then all line endings will be converted to '\n' as described for the universal newlines 'U' mode argument to open().

###### shell
If shell is True, the specified command will be executed through the shell.   
This can be useful if you are using Python primarily for the enhanced control flow it offers over most system shells and still want convenient access to other shell features such as shell pipes, filename wildcards, environment variable expansion, and expansion of ~ to a user’s home directory.   
However, note that Python itself offers implementations of many shell-like features (in particular, glob, fnmatch, os.walk(), os.path.expandvars(), os.path.expanduser(), and shutil).  
Warning:  
Executing shell commands that incorporate sanitized input from an untrusted source makes a program vulnerable to shell injection, a serious security flaw which can result in arbitrary command execution.   
For this reason, the use of shell=True is strongly discouraged in cases where the command string is constructed from external input:  
>>> from subprocess import call  
>>> filename = input("What file would you like to display?\n")  
What file would you like to display?  
non_existent; rm -rf / #  
>>> call("cat " + filename, shell=True) # Uh-oh. This will end badly...  

shell=False disables all shell based features, but does not suffer from this vulnerability;   
see the Note in the Popen constructor documentation for helpful hints in getting shell=False to work.  

When using shell=True, pipes.quote() can be used to properly escape whitespace and shell metacharacters in strings that are going to be used to construct shell commands.

### Popen Constructor

The underlying process creation and management in this module is handled by the Popen class.   
It offers a lot of flexibility so that developers are able to handle the less common cases not covered by the convenience functions.

###### class subprocess.Popen(args, bufsize=0, executable=None, stdin=None, stdout=None, stderr=None, preexec_fn=None, close_fds=False, shell=False, cwd=None, env=None, universal_newlines=False, startupinfo=None, creationflags=0)
Execute a child program in a new process.   
On Unix, the class uses os.execvp()-like behavior to execute the child program.   
On Windows, the class uses the Windows CreateProcess() function.   
The arguments to Popen are as follows:

###### args
args should be a sequence of program arguments or else a single string.   
By default, the program to execute is the first item in args if args is a sequence.   
If args is a string, the interpretation is platform-dependent and described below.   
See the shell and executable arguments for additional differences from the default behavior.   
Unless otherwise stated, it is recommended to pass args as a sequence.  

On Unix, if args is a string, the string is interpreted as the name or path of the program to execute.   
However, this can only be done if not passing arguments to the program.  
Note:   
shlex.split() can be useful when determining the correct tokenization for args, especially in complex cases:

In [20]:
import shlex, subprocess
command_line = 'mysql -uroot -S /tmp/mysql.sock -e "show databases;"'
args = shlex.split(command_line)
print args

['mysql', '-uroot', '-S', '/tmp/mysql.sock', '-e', 'show databases;']


In [25]:
p = subprocess.Popen(args)

###### shell
The shell argument (which defaults to False) specifies whether to use the shell as the program to execute.   
If shell is True, it is recommended to pass args as a string rather than as a sequence.

On Unix with shell=True, the shell defaults to /bin/sh.   
If args is a string, the string specifies the command to execute through the shell.   
This means that the string must be formatted exactly as it would be when typed at the shell prompt.   
This includes, for example, quoting or backslash escaping filenames with spaces in them.   
If args is a sequence, the first item specifies the command string, and any additional items will be treated as additional arguments to the shell itself.   
That is to say, Popen does the equivalent of:  
>Popen(['/bin/sh', '-c', args[0], args[1], ...])  

Warning:  
Passing shell=True can be a security hazard if combined with untrusted input.   
See the warning under Frequently Used Arguments for details.

###### bufsize
bufsize, if given, has the same meaning as the corresponding argument to the built-in open() function: 0 means unbuffered, 1 means line buffered, any other positive value means use a buffer of (approximately) that size.   
A negative bufsize means to use the system default, which usually means fully buffered.   
The default value for bufsize is 0 (unbuffered).  
Note:  
If you experience performance issues, it is recommended that you try to enable buffering by setting bufsize to either -1 or a large enough positive value (such as 4096).  

###### executable 
The executable argument specifies a replacement program to execute.   
It is very seldom needed.   
When shell=False, executable replaces the program to execute specified by args.   
However, the original args is still passed to the program.   
Most programs treat the program specified by args as the command name, which can then be different from the program actually executed.   
On Unix, the args name becomes the display name for the executable in utilities such as ps.   
If shell=True, on Unix the executable argument specifies a replacement shell for the default /bin/sh.

###### stdin, stdout and stderr
stdin, stdout and stderr specify the executed program’s standard input, standard output and standard error file handles, respectively.   
Valid values are PIPE, an existing file descriptor (a positive integer), an existing file object, and None.   
PIPE indicates that a new pipe to the child should be created.   
With the default settings of None, no redirection will occur; the child’s file handles will be inherited from the parent.   
Additionally, stderr can be STDOUT, which indicates that the stderr data from the child process should be captured into the same file handle as for stdout.

###### preexec_fn 
If preexec_fn is set to a callable object, this object will be called in the child process just before the child is executed. (Unix only)

###### close_fds 
If close_fds is true, all file descriptors except 0, 1 and 2 will be closed before the child process is executed.(Unix only).

###### cwd 
If cwd is not None, the child’s current directory will be changed to cwd before it is executed.   
Note that this directory is not considered when searching the executable, so you can’t specify the program’s path relative to cwd.  

###### env 
If env is not None, it must be a mapping that defines the environment variables for the new process; these are used instead of inheriting the current process’ environment, which is the default behavior.  
Note:  
If specified, env must provide any variables required for the program to execute. 

###### universal_newlines 
If universal_newlines is True, the file objects stdout and stderr are opened as text files in universal newlines mode.   
Lines may be terminated by any of '\n', the Unix end-of-line convention, '\r', the old Macintosh convention or '\r\n', the Windows convention.   
All of these external representations are seen as '\n' by the Python program.  
Note:  
This feature is only available if Python is built with universal newline support (the default).   
Also, the newlines attribute of the file objects stdout, stdin and stderr are not updated by the communicate() method.

###### startupinfo 
If given, startupinfo will be a STARTUPINFO object, which is passed to the underlying CreateProcess function.  

###### creationflags
creationflags, if given, can be CREATE_NEW_CONSOLE or CREATE_NEW_PROCESS_GROUP. (Windows only)

### Exceptions

Exceptions raised in the child process, before the new program has started to execute, will be re-raised in the parent.   
Additionally, the exception object will have one extra attribute called child_traceback, which is a string containing traceback information from the child’s point of view.  

###### OSError
The most common exception raised is OSError.   
This occurs, for example, when trying to execute a non-existent file.   
Applications should prepare for OSError exceptions.

###### ValueError
A ValueError will be raised if Popen is called with invalid arguments.

###### CalledProcessError
check_call() and check_output() will raise CalledProcessError if the called process returns a non-zero return code.

### Security

Unlike some other popen functions, this implementation will never call a system shell implicitly.   
This means that all characters, including shell metacharacters, can safely be passed to child processes.   Obviously, if the shell is invoked explicitly, then it is the application’s responsibility to ensure that all whitespace and metacharacters are quoted appropriately.